# 1 - Présentation
## 1.1 - Introduction
Il est possible de récupérer des données sur les réseaux sociaux et d'autres sources de manières exponentielles. Il n'est pas envisageable qu'un être humain utilise une grande partie de son temps à les traiter.

Le but de ce projet est de récupérer des informations sur des évènements venant de Facebook et d'appliquer des classifications sur ces informations de manière automatique.

L'objectif est d'entrainer un programme à lire un évènement et à appliquer des classifications.

## 1.2 - Source de Données
Pour l'exercice la source sera un fichier texte que j'aurai écris avec des évènemenents que j'aurai récupérés sur Facebook.

# 2 - Préparation des données
## 2.1 - Charger les librairies

In [23]:
import pandas as pd

## 2.2 - Charger les données

In [21]:
source = [
    ["Soirée Electro samedi soir au hangar à banane", "party"],
    ["Soirée Cinéma vendredi soir au UGC à Nantes pour voir le film Trolls", "movies"],
    ["Electro Bitch à la rochelle", "party"],
    ["",""]
]

[[u'Soir\xe9e Electro samedi soir au hangar \xe0 banane', u'party'],
 [u'Soir\xe9e Cin\xe9ma vendredi soir au UGC \xe0 Nantes pour voir le film Trolls',
  u'movies'],
 [u'Electro Bitch \xe0 la rochelle', u'party'],
 [u'', u'']]

## 2.3 - Nettoyer les données

In [22]:
data

[[u'Soir\xe9e Electro samedi soir au hangar \xe0 banane', u'party'],
 [u'Soir\xe9e Cin\xe9ma vendredi soir au UGC \xe0 Nantes pour voir le film Trolls',
  u'movies'],
 [u'Electro Bitch \xe0 la rochelle', u'party'],
 [u'', u'']]